In [ ]:
# upload whisper_dataset file
from google.colab import drive
drive.mount('/content/drive')

import shutil
shutil.copy("YOUR_PATH_TO_whisper_dataset_INSIDE_DRIVE","/content/")

In [1]:
import json

with open("config.json",'r') as file:
    json_object = json.load(file)
    filename = json_object["whisper_dataset"]

In [2]:
!pip install -q transformers
!pip install -q spacy
!pip install -q sentencepiece
!pip install -q bert-extractive-summarizer

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objs as go


## using BERT based model EmoRoBERTa for sententiment detection 

In [4]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline

tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [5]:
text = ""
df = pd.read_csv(filename)
df = df[["Speakers","Texts"]]
for idx in range(len(df)):
    text=text+"  "+df.iloc[idx,1]

In [6]:
df1 = df.copy(deep=True)
emotions = []

for idx in tqdm(range(len(df1))):
    speech = df1.iloc[idx,1]
    emotion_labels = emotion(speech)
    emotions.append(emotion_labels[0]["label"])
    
mapping = {
    'neutral' : 'neutral',
    'admiration': 'happiness',
    'amusement': 'happiness',
    'anger': 'anger',
    'annoyance': 'anger',
    'approval': 'happiness',
    'caring': 'sadness',
    'confusion': 'surprise',
    'curiosity': 'surprise',
    'desire': 'happiness',
    'disappointment': 'sadness',
    'disapproval': 'anger',
    'disgust': 'disgust',
    'embarrassment': 'disgust',
    'excitement': 'happiness',
    'fear': 'fear',
    'gratitude': 'happiness',
    'grief': 'sadness',
    'joy': 'happiness',
    'love': 'happiness',
    'nervousness': 'fear',
    'optimism': 'happiness',
    'pride': 'happiness',
    'realization': 'surprise',
    'relief': 'happiness',
    'remorse': 'sadness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}

emotions = [*map(mapping.get,emotions)]
df1["Emotions"] = np.array(emotions)
df1 = df1.groupby(['Speakers', 'Emotions']).size().reset_index(name='Counts')

100%|██████████| 220/220 [01:11<00:00,  3.06it/s]


In [7]:
color_scale = {
    'neutral': 'whitesmoke',
    'happiness': 'springgreen',
    'surprise': 'torquise',
    'anger': 'crimson',
    'sadness': 'black',
    'disgust': 'darkviolet'
}

fig = px.pie(df1, values='Counts', names='Emotions', color='Emotions', 
             color_discrete_map=color_scale,color_discrete_sequence=px.colors.qualitative.Pastel,
             labels={'Emotions': 'Emotion', 'Counts': 'Counts', 'Speakers': 'Speakers'},
             title='BERT Emotion Distribution by Speaker')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()


In [8]:
fig = px.scatter_3d(df1, x='Speakers', y='Emotions', z='Counts', color='Speakers', 
                    size='Counts', opacity=0.8)

fig.update_layout(scene=dict(xaxis_title='Speakers', yaxis_title='Emotions', zaxis_title='Counts'))

fig.show()

## text summarization using BERT

In [9]:
from summarizer import Summarizer

model = Summarizer()
summary = ''.join(model(text,min_length=10,max_length=100))


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning:

Can't initialize NVML

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/sklearn/clu

In [10]:
print("summary using BERT \n")
print("=================================================")
print(summary)
print("=================================================")

summary using BERT 

April 2nd 2019. Okay, okay. So if I change like...  Oh yeah, yeah. I'll just take your word for it. Let me guess the number. I thought it was one output. Is it an odd number or even number? And you still fucked up. In the above code, what is the main difference between  There's a minus there. What you think number 9 is? What's 5 and 6? No, 5 is false. Why did you put A for the last one? No, I put B.  Oh, then who said it was A? Oh, you are. I don't know how, but you will. This array will get 10 names from the user. Well, I mean, lab ends at 530. And then you can put in three names. I don't know. Do you understand it? Before the four statement because it's going to do that ten times too. Other than that, that's good. You heard me. I got shit to do. Yeah, I'm saving people. It's a bad school. You have good food. Like the school I visited, they were always the school I visited there. No, I've heard people. No, my shirt won't work with your shirt. Because you don't hav

## Topic detection on generated summary of BERT

In [12]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

mname = "cristian-popa/bart-tl-ng"
tokenizer = AutoTokenizer.from_pretrained(mname)
model = AutoModelForSeq2SeqLM.from_pretrained(mname)

enc = tokenizer(summary, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
outputs = model.generate(
    input_ids=enc.input_ids,
    attention_mask=enc.attention_mask,
    max_length=15,
    min_length=1,
    do_sample=False,
    num_beams=25,
    length_penalty=1.0,
    repetition_penalty=1.5
)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [13]:
print(decoded)

the difference


## using T5 Transformer based model for sentiment detection

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-emotion")


In [15]:
text = ""
df = pd.read_csv(filename)
df = df[["Speakers","Texts"]]
for idx in range(len(df)):
    text=text+"  "+df.iloc[idx,1]


In [16]:
import re

df1 = df.copy(deep=True)
emotions = []

for idx in tqdm(range(len(df1))):
    speech = df1.iloc[idx,1]
    input_ids = tokenizer.encode(speech, return_tensors='pt')
    output = model.generate(input_ids=input_ids,max_length=2)
    dec = [tokenizer.decode(ids) for ids in output]
    label = re.sub('<pad>','',dec[0])
    emotions.append(label.strip())

    mapping = {
    'neutral' : 'neutral',
    'admiration': 'happiness',
    'amusement': 'happiness',
    'anger': 'anger',
    'annoyance': 'anger',
    'approval': 'happiness',
    'caring': 'sadness',
    'confusion': 'surprise',
    'curiosity': 'surprise',
    'desire': 'happiness',
    'disappointment': 'sadness',
    'disapproval': 'anger',
    'disgust': 'disgust',
    'embarrassment': 'disgust',
    'excitement': 'happiness',
    'fear': 'fear',
    'gratitude': 'happiness',
    'grief': 'sadness',
    'joy': 'happiness',
    'love': 'happiness',
    'nervousness': 'fear',
    'optimism': 'happiness',
    'pride': 'happiness',
    'realization': 'surprise',
    'relief': 'happiness',
    'remorse': 'sadness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}

emotions = [*map(mapping.get,emotions)]
df1["Emotions"] = np.array(emotions)
df1 = df1.groupby(['Speakers', 'Emotions']).size().reset_index(name='Counts')

100%|██████████| 220/220 [00:44<00:00,  4.93it/s]


In [17]:
color_scale = {
    'neutral': 'whitesmoke',
    'happiness': 'springgreen',
    'surprise': 'torquise',
    'anger': 'crimson',
    'sadness': 'black',
    'disgust': 'darkviolet'
}

fig = px.pie(df1, values='Counts', names='Emotions', color='Emotions', color_discrete_map=color_scale,
             labels={'Emotions': 'Emotion', 'Counts': 'Counts', 'Speakers': 'Speakers'},
             title='T5 Emotion Distribution by Speaker')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()


In [18]:
fig = px.scatter_3d(df1, x='Speakers', y='Emotions', z='Counts', color='Speakers', 
                    size='Counts', opacity=0.8)

fig.update_layout(scene=dict(xaxis_title='Speakers', yaxis_title='Emotions', zaxis_title='Counts'))

fig.show()

## Text summarization using T5

In [19]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
text = text.strip().replace("\n","")
enc = tokenizer.encode(text,return_tensors="pt")
summary_ids = model.generate(enc,num_beams=4,no_repeat_ngram_size=2,min_length=30,max_length=100,early_stopping=True)
summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning:

This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.

Token indices sequence length is longer than the specified maximum sequence length for this model (1682 > 512). Running this sequence through the model will result in indexing errors


In [20]:
print("summary using T5 \n")
print("=================================================")
print(summary)
print("=================================================")

summary using T5 

it. I'm gonna betraying me. You're going to search it with the key. Yeah. Okay, okay. So you've got a question wrong. That's what you say. And then you can do array. To stream. 47. 48. This is specifically asking for the index number. No, like is it because like num takes in B? Yes, I think. But you have to do it ten times too.


## Topic Detection on generated summary of T5

In [21]:
from transformers import AutoModelForSeq2SeqLM

mname = "cristian-popa/bart-tl-ng"
tokenizer = AutoTokenizer.from_pretrained(mname)
model = AutoModelForSeq2SeqLM.from_pretrained(mname)

enc = tokenizer(summary, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
outputs = model.generate(
    input_ids=enc.input_ids,
    attention_mask=enc.attention_mask,
    max_length=15,
    min_length=1,
    do_sample=False,
    num_beams=25,
    length_penalty=1.0,
    repetition_penalty=1.5
)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)



In [22]:
print(decoded)

sort of


## using XLNet Transformer based model for sentiment detection

In [23]:
from transformers import XLNetForSequenceClassification,XLNetTokenizer

model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [24]:
text = ""
df = pd.read_csv(filename)
df = df[["Speakers","Texts"]]
for idx in range(len(df)):
    text=text+"  "+df.iloc[idx,1]


In [25]:
df1 = df.copy(deep=True)
emotions = []

for idx in tqdm(range(len(df1))):
    speech = df1.iloc[idx,1]
    input_ids = tokenizer.encode(speech,add_special_tokens=True,return_tensors='pt')
    output = model(input_ids=input_ids,max_length=2)
    dec = output.logits.argmax().item()
    label = {0:"negetive",1:"positive"}
    emotions.append(label[dec])
    
df1["Emotions"] = np.array(emotions)
df1 = df1.groupby(['Speakers', 'Emotions']).size().reset_index(name='Counts')

100%|██████████| 220/220 [00:37<00:00,  5.87it/s]


In [26]:
color_scale = {
    'positive': 'whitesmoke',
    'negative': 'crimson'
}

fig = px.pie(df1, values='Counts', names='Emotions', color='Emotions', color_discrete_map=color_scale,
             labels={'Emotions': 'Emotion', 'Counts': 'Counts', 'Speakers': 'Speakers'},
             title='XLNet Emotion Distribution by Speaker')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()

In [27]:
fig = px.scatter_3d(df1, x='Speakers', y='Emotions', z='Counts', color='Speakers', 
                    size='Counts', opacity=0.8)

fig.update_layout(scene=dict(xaxis_title='Speakers', yaxis_title='Emotions', zaxis_title='Counts'))

fig.show()

## text summarization using XLNet

In [28]:
from summarizer import TransformerSummarizer

model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
summary = ''.join(model(text,min_length=60))

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [29]:
print("summary using XLNet \n")
print("================================")
print(summary)
print("================================")

summary using XLNet 

I thought I meant like changing it to another thing, whatever. I'm talking about 4 because it's a public static void method. Let's get this class activity because I already didn't turn in the other one. Before the four statement because it's going to do that ten times too.


## Topic Detection on generated summary of XLNet

In [30]:
from transformers import AutoModelForSeq2SeqLM

mname = "cristian-popa/bart-tl-ng"
tokenizer = AutoTokenizer.from_pretrained(mname)
model = AutoModelForSeq2SeqLM.from_pretrained(mname)

enc = tokenizer(summary, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
outputs = model.generate(
    input_ids=enc.input_ids,
    attention_mask=enc.attention_mask,
    max_length=15,
    min_length=1,
    do_sample=False,
    num_beams=25,
    length_penalty=1.0,
    repetition_penalty=1.5
)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)



In [31]:
print(decoded)

addition


In [32]:
#